In [ ]:
import torch 
from diffusers import DPMSolverMultistepScheduler, StableDiffusionInpaintPipeline
from PIL import Image
from torch import autocast
device = "cuda" if torch.cuda.is_available() else "cpu"
import numpy as np 

In [2]:
model_path = "RealVis"

num_inference_steps = 20

guidance_scale = 10

strength = 1

clip_slip = 1

padding_mask_crop = 7

generator = torch.Generator("cuda").manual_seed(3119)

# width = 512 
# height = 512 

prompt = "remove text, preserve background, highres, quality" 

negative_prompt = "deformed, weird, bad resolution, bad depiction, blurry, not relevant"


In [ ]:
scheduler = DPMSolverMultistepScheduler.from_pretrained(model_path, subfolder="scheduler", use_karras_sigmas=True)

pipeline = StableDiffusionInpaintPipeline.from_pretrained(
    pretrained_model_name_or_path = model_path,
    torch_dtype = torch.float16,
    variant = "fp16",
    safety_checker=None,
    scheduler=scheduler,
)
pipeline.enable_model_cpu_offload()

In [5]:
image_name = "1003001137771826"
image_path = f"./test_image/{image_name}.png"
mask_path  = f"./test_image/{image_name}_mask.png"

img  = Image.open(image_path).convert("RGB")
mask = Image.open(mask_path).convert("L")


orig_width, orig_height = img.size

# img = img.resize((width, height))
# mask = mask.resize((width, height))


In [ ]:
with autocast("cuda"):
    result = pipeline(prompt=prompt,
                      image=img, 
                      mask_image=mask,
                      num_inference_steps=num_inference_steps, 
                      strength=strength, 
                      guidance_scale=guidance_scale,
                      generator=generator,
                      width=orig_width,
                      height=orig_height,
                      clip_skip=clip_slip,
                      padding_mask_crop=padding_mask_crop,).images[0]

In [7]:
final = Image.composite(result, img, mask)
final.save(f"./test_image/{image_name}_inpainted.png")